In [1]:
# conda activate snakepipes_detect-seq
# python -m ipykernel install --user --name snakepipes_detect-seq
# 接下来使用这个ipykernel进行分析，所有依赖包已经在环境中装好了

In [2]:
# %% library
from glob import glob
import os
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import multipletests

# %%% pandas settings
# pd.set_option("max_colwidth", 200)  # column最大宽度
pd.set_option("display.width", 200)  # dataframe宽度
pd.set_option("display.max_columns", None)  # column最大显示数
pd.set_option("display.max_rows", 50)  # row最大显示数

# 初步过滤 poisson_res

In [3]:
# 此表由snakepipes_detect-seq, step.06.Call-Off-target.py所得！！！
file = '../poisson_res/poisson_res_all.tsv.gz'
df = pd.read_csv(file, header=0, index_col=None, sep='\t', engine='pyarrow')
df.head()

,<sample>,chr_name,region_start,region_end,mpmat_index,region_site_num,region_block_site_num,region_mut_site_num,region_site_index,region_block_state,region_highest_site_index,region_highest_site_mut_num,region_highest_site_cover_num,region_highest_site_mut_ratio,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR
0,DetectSeq_ATP8-DddA11_REP-1,chr1,70287,70302,chr1_70287_70302,7,0,7,"chr1_70287_CT,chr1_70288_C.,chr1_70291_CT,chr1...",N-N-N-N-N-N-N,chr1_70300_CT,5,15,0.333333,16,21,0,7,0.040504,0.125178,0.0,0.041726,"0,1,2,3,4,5,6,7 15,1,0,0,0,0,0,0 12,2,5,2,0,0,0,0",1.627834,2.364909,TestOK,0.024256,0.445138
1,DetectSeq_ATP8-DddA11_REP-1,chr1,191793,191812,chr1_191793_191812,7,4,3,"chr1_191793_CT,chr1_191798_C.,chr1_191799_CT,c...",N-B-B-N-B-N-B,chr1_191803_CT,2,11,0.181818,36,13,0,1,0.091135,0.077491,0.0,0.005961,"0,1,2,3,4,5,6,7 33,3,0,0,0,0,0,0 11,1,1,0,0,0,0,0",-0.233969,-0.442445,TestOK,0.582932,0.842856
2,DetectSeq_ATP8-DddA11_REP-1,chr1,262770,262804,chr1_262770_262804,4,0,4,"chr1_262770_GA,chr1_262789_GA,chr1_262798_GA,c...",N-N-N-N,chr1_262770_GA,0,12,0.000000,6,14,0,0,0.015189,0.083452,0.0,0.000000,"0,1,2,3,4 6,0,0,0,0 14,0,0,0,0",2.457909,NaN,TestOK,0.841752,0.842856
3,DetectSeq_ATP8-DddA11_REP-1,chr1,268547,268579,chr1_268547_268579,10,0,10,"chr1_268547_GA,chr1_268549_G.,chr1_268550_GA,c...",N-N-N-N-N-N-N-N-N-N,chr1_268547_GA,0,5,0.000000,43,6,0,0,0.108855,0.035765,0.0,0.000000,"0,1,2,3,4,5,6,7,8,9,10 37,6,0,0,0,0,0,0,0,0,0 ...",-1.605786,NaN,TestOK,0.841752,0.842856
4,DetectSeq_ATP8-DddA11_REP-1,chr1,350659,350665,chr1_350659_350665,3,0,3,"chr1_350659_GA,chr1_350660_GA,chr1_350665_GA",N-N-N,chr1_350659_GA,4,7,0.571429,0,7,0,3,0.000000,0.041726,0.0,0.017883,"0,1,2,3 0,0,0,0 3,0,4,0",-0.649206,1.142517,TestOK,0.232584,0.842856


In [4]:
# check columns
df.columns

Index(['<sample>', 'chr_name', 'region_start', 'region_end', 'mpmat_index', 'region_site_num', 'region_block_site_num', 'region_mut_site_num', 'region_site_index', 'region_block_state',
       'region_highest_site_index', 'region_highest_site_mut_num', 'region_highest_site_cover_num', 'region_highest_site_mut_ratio', 'ctrl_count', 'treat_count', 'ctrl_mut_count', 'treat_mut_count',
       'ctrl_count.norm', 'treat_count.norm', 'ctrl_mut_count.norm', 'treat_mut_count.norm', 'count_info', 'log2_FC', 'log2_FC_mut', 'test_state', 'p_value', 'FDR'],
      dtype='object')

In [5]:
# %%% check table and filter test
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705649 entries, 0 to 705648
Data columns (total 28 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   <sample>                       705649 non-null  object 
 1   chr_name                       705649 non-null  object 
 2   region_start                   705649 non-null  int64  
 3   region_end                     705649 non-null  int64  
 4   mpmat_index                    705649 non-null  object 
 5   region_site_num                705649 non-null  int64  
 6   region_block_site_num          705649 non-null  int64  
 7   region_mut_site_num            705649 non-null  int64  
 8   region_site_index              705649 non-null  object 
 9   region_block_state             705649 non-null  object 
 10  region_highest_site_index      705649 non-null  object 
 11  region_highest_site_mut_num    705649 non-null  int64  
 12  region_highest_site_cover_num 

In [6]:
df.isna().sum()
# log2_FC                            1010
# log2_FC_mut                      404057

<sample>                              0
chr_name                              0
region_start                          0
region_end                            0
mpmat_index                           0
region_site_num                       0
region_block_site_num                 0
region_mut_site_num                   0
region_site_index                     0
region_block_state                    0
region_highest_site_index             0
region_highest_site_mut_num           0
region_highest_site_cover_num         0
region_highest_site_mut_ratio         0
ctrl_count                            0
treat_count                           0
ctrl_mut_count                        0
treat_mut_count                       0
ctrl_count.norm                       0
treat_count.norm                      0
ctrl_mut_count.norm                   0
treat_mut_count.norm                  0
count_info                            0
log2_FC                            1010
log2_FC_mut                      404057


In [7]:
# filter test sample NAN应该是 test sample 中的，过滤一下
df = df.query("`<sample>` != 'test'")
df['<sample>'].unique()

array(['DetectSeq_ATP8-DddA11_REP-1', 'DetectSeq_ATP8-DddA6_REP-1',
       'DetectSeq_ATP8-DddAwt_REP-1', 'DetectSeq_JAK2-DddA11_REP-1',
       'DetectSeq_JAK2-DddA11_REP-2', 'DetectSeq_SIRT6-DddA11_REP-1',
       'DetectSeq_SIRT6-DddA11_REP-2'], dtype=object)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 705649 entries, 0 to 705648
Data columns (total 28 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   <sample>                       705649 non-null  object 
 1   chr_name                       705649 non-null  object 
 2   region_start                   705649 non-null  int64  
 3   region_end                     705649 non-null  int64  
 4   mpmat_index                    705649 non-null  object 
 5   region_site_num                705649 non-null  int64  
 6   region_block_site_num          705649 non-null  int64  
 7   region_mut_site_num            705649 non-null  int64  
 8   region_site_index              705649 non-null  object 
 9   region_block_state             705649 non-null  object 
 10  region_highest_site_index      705649 non-null  object 
 11  region_highest_site_mut_num    705649 non-null  int64  
 12  region_highest_site_cover_num  7056

In [9]:
df.isna().sum()  # 过滤掉 test sample 之后 NAN 少了很多
# log2_FC                            1010
# log2_FC_mut                      404057

<sample>                              0
chr_name                              0
region_start                          0
region_end                            0
mpmat_index                           0
region_site_num                       0
region_block_site_num                 0
region_mut_site_num                   0
region_site_index                     0
region_block_state                    0
region_highest_site_index             0
region_highest_site_mut_num           0
region_highest_site_cover_num         0
region_highest_site_mut_ratio         0
ctrl_count                            0
treat_count                           0
ctrl_mut_count                        0
treat_mut_count                       0
ctrl_count.norm                       0
treat_count.norm                      0
ctrl_mut_count.norm                   0
treat_mut_count.norm                  0
count_info                            0
log2_FC                            1010
log2_FC_mut                      404057


In [10]:
# add highest index info
# df = df.assign(bed_name=df.mpmat_index + '_highest_' + df.region_highest_site_index, strand='.')
# df['bed_name']
# df['bed_name'][1]


# check mpmat_index所有样本一致性
df.groupby('<sample>').mpmat_index.describe()
# 因为前期 call 点的时候，把所有的 region 放在一起考虑的
# 这里mpmat_index unique，即 frequency == 1 才合理

#                                count  unique               top freq
# <sample>
# DetectSeq_ATP8-DddA11_REP-1   100807  100807  chr1_70287_70302    1
# DetectSeq_ATP8-DddA6_REP-1    100807  100807  chr1_70287_70302    1
# DetectSeq_ATP8-DddAwt_REP-1   100807  100807  chr1_70287_70302    1
# DetectSeq_JAK2-DddA11_REP-1   100807  100807  chr1_70287_70302    1
# DetectSeq_JAK2-DddA11_REP-2   100807  100807  chr1_70287_70302    1
# DetectSeq_SIRT6-DddA11_REP-1  100807  100807  chr1_70287_70302    1
# DetectSeq_SIRT6-DddA11_REP-2  100807  100807  chr1_70287_70302    1
# call 点没问题，继续后续分析

,count,unique,top,freq
<sample>,,,,
DetectSeq_ATP8-DddA11_REP-1,100807,100807,chr1_70287_70302,1
DetectSeq_ATP8-DddA6_REP-1,100807,100807,chr1_70287_70302,1
DetectSeq_ATP8-DddAwt_REP-1,100807,100807,chr1_70287_70302,1
DetectSeq_JAK2-DddA11_REP-1,100807,100807,chr1_70287_70302,1
DetectSeq_JAK2-DddA11_REP-2,100807,100807,chr1_70287_70302,1
DetectSeq_SIRT6-DddA11_REP-1,100807,100807,chr1_70287_70302,1
DetectSeq_SIRT6-DddA11_REP-2,100807,100807,chr1_70287_70302,1


In [11]:
# %%% 重算FDR 如果需要的话，平时不需要
# for sample, tmp_s in df.groupby('<sample>')['p_value']:
#     df.loc[df['<sample>'] == sample, 'FDR'] = multipletests(tmp_s, alpha=0.05, method="fdr_bh", is_sorted=False)[1]


# df['FDR'].isna().sum()

## find significant region - strict selection

In [12]:
df

,<sample>,chr_name,region_start,region_end,mpmat_index,region_site_num,region_block_site_num,region_mut_site_num,region_site_index,region_block_state,region_highest_site_index,region_highest_site_mut_num,region_highest_site_cover_num,region_highest_site_mut_ratio,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR
0,DetectSeq_ATP8-DddA11_REP-1,chr1,70287,70302,chr1_70287_70302,7,0,7,"chr1_70287_CT,chr1_70288_C.,chr1_70291_CT,chr1...",N-N-N-N-N-N-N,chr1_70300_CT,5,15,0.333333,16,21,0,7,0.040504,0.125178,0.0,0.041726,"0,1,2,3,4,5,6,7 15,1,0,0,0,0,0,0 12,2,5,2,0,0,0,0",1.627834,2.364909,TestOK,0.024256,0.445138
1,DetectSeq_ATP8-DddA11_REP-1,chr1,191793,191812,chr1_191793_191812,7,4,3,"chr1_191793_CT,chr1_191798_C.,chr1_191799_CT,c...",N-B-B-N-B-N-B,chr1_191803_CT,2,11,0.181818,36,13,0,1,0.091135,0.077491,0.0,0.005961,"0,1,2,3,4,5,6,7 33,3,0,0,0,0,0,0 11,1,1,0,0,0,0,0",-0.233969,-0.442445,TestOK,0.582932,0.842856
2,DetectSeq_ATP8-DddA11_REP-1,chr1,262770,262804,chr1_262770_262804,4,0,4,"chr1_262770_GA,chr1_262789_GA,chr1_262798_GA,c...",N-N-N-N,chr1_262770_GA,0,12,0.000000,6,14,0,0,0.015189,0.083452,0.0,0.000000,"0,1,2,3,4 6,0,0,0,0 14,0,0,0,0",2.457909,NaN,TestOK,0.841752,0.842856
3,DetectSeq_ATP8-DddA11_REP-1,chr1,268547,268579,chr1_268547_268579,10,0,10,"chr1_268547_GA,chr1_268549_G.,chr1_268550_GA,c...",N-N-N-N-N-N-N-N-N-N,chr1_268547_GA,0,5,0.000000,43,6,0,0,0.108855,0.035765,0.0,0.000000,"0,1,2,3,4,5,6,7,8,9,10 37,6,0,0,0,0,0,0,0,0,0 ...",-1.605786,NaN,TestOK,0.841752,0.842856
4,DetectSeq_ATP8-DddA11_REP-1,chr1,350659,350665,chr1_350659_350665,3,0,3,"chr1_350659_GA,chr1_350660_GA,chr1_350665_GA",N-N-N,chr1_350659_GA,4,7,0.571429,0,7,0,3,0.000000,0.041726,0.0,0.017883,"0,1,2,3 0,0,0,0 3,0,4,0",-0.649206,1.142517,TestOK,0.232584,0.842856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705644,DetectSeq_SIRT6-DddA11_REP-2,chrX,155682866,155682903,chrX_155682866_155682903,10,0,10,"chrX_155682866_GA,chrX_155682868_GA,chrX_15568...",N-N-N-N-N-N-N-N-N-N,chrX_155682874_GA,1,10,0.100000,17,12,0,1,0.043036,0.055731,0.0,0.004644,"0,1,2,3,4,5,6,7,8,9,10 16,1,0,0,0,0,0,0,0,0,0 ...",0.372935,-0.534133,TestOK,0.577542,0.896841
705645,DetectSeq_SIRT6-DddA11_REP-2,chrX,155698826,155698840,chrX_155698826_155698840,4,0,4,"chrX_155698826_GA,chrX_155698835_G.,chrX_15569...",N-N-N-N,chrX_155698826_GA,0,15,0.000000,23,16,0,0,0.058225,0.074307,0.0,0.000000,"0,1,2,3,4 23,0,0,0,0 16,0,0,0,0",0.351873,NaN,TestOK,0.838398,0.896841
705646,DetectSeq_SIRT6-DddA11_REP-2,chrX,155698884,155698890,chrX_155698884_155698890,2,0,2,"chrX_155698884_GA,chrX_155698890_GA",N-N,chrX_155698884_GA,0,12,0.000000,32,12,0,0,0.081008,0.055731,0.0,0.000000,"0,1,2 30,2,0 12,0,0",-0.539603,NaN,TestOK,0.838398,0.896841
705647,DetectSeq_SIRT6-DddA11_REP-2,chrX,155836011,155836014,chrX_155836011_155836014,3,0,3,"chrX_155836011_CT,chrX_155836012_CT,chrX_15583...",N-N-N,chrX_155836011_CT,0,0,0.000000,0,0,0,0,0.000000,0.000000,0.0,0.000000,"0,1,2,3 0,0,0,0 0,0,0,0",NaN,NaN,TestOK,0.838398,0.896841


In [13]:
df_sign_strict = (
    df
    .query('FDR <= 0.0001')
    # .query('p_value <= 0.0001')
    .query('log2_FC_mut >= 2')
    .query('ctrl_mut_count <= 1')
    .query('`treat_mut_count.norm` * 100 >= 10')
    .query('treat_mut_count >= 20')
    .query('treat_mut_count / treat_count >= 0.15')
    .query('region_block_site_num <= 1')
    .query('region_highest_site_mut_ratio >= 0.35')
)

print(df_sign_strict.shape[0])
print(df_sign_strict.groupby('<sample>').mpmat_index.count())
print(df_sign_strict.isna().sum().sum())  # check是否有空值
# <sample>
# DetectSeq_ATP8-DddA11_REP-1      101
# DetectSeq_ATP8-DddA6_REP-1       328
# DetectSeq_ATP8-DddAwt_REP-1       59
# DetectSeq_JAK2-DddA11_REP-1      568
# DetectSeq_JAK2-DddA11_REP-2      314
# DetectSeq_SIRT6-DddA11_REP-1    1638
# DetectSeq_SIRT6-DddA11_REP-2    3106
# Name: mpmat_index, dtype: int64

6114
<sample>
DetectSeq_ATP8-DddA11_REP-1      101
DetectSeq_ATP8-DddA6_REP-1       328
DetectSeq_ATP8-DddAwt_REP-1       59
DetectSeq_JAK2-DddA11_REP-1      568
DetectSeq_JAK2-DddA11_REP-2      314
DetectSeq_SIRT6-DddA11_REP-1    1638
DetectSeq_SIRT6-DddA11_REP-2    3106
Name: mpmat_index, dtype: int64
0


## find significant region - lenient selection

In [14]:
# nature protocol condicitons
df_sign_lenient = (
    df
    .query('FDR <= 0.01')
    # .query('p_value <= 0.0005')
    .query('log2_FC_mut >= 2')
    .query('ctrl_mut_count <= 1')
    .query('`treat_mut_count.norm` * 100 >= 5')
    .query('treat_mut_count >= 10')
    .query('treat_mut_count / treat_count >= 0.15')
    .query('region_block_site_num <= 1')
    .query('region_highest_site_mut_ratio >= 0.30')
)
print(df_sign_lenient.shape[0])
print(df_sign_lenient.groupby('<sample>').mpmat_index.count())
print(df_sign_lenient.isna().sum().sum())  # check是否有空值
# <sample>
# DetectSeq_ATP8-DddA11_REP-1      307
# DetectSeq_ATP8-DddA6_REP-1       931
# DetectSeq_ATP8-DddAwt_REP-1      145
# DetectSeq_JAK2-DddA11_REP-1     1014
# DetectSeq_JAK2-DddA11_REP-2      874
# DetectSeq_SIRT6-DddA11_REP-1    3888
# DetectSeq_SIRT6-DddA11_REP-2    9673
# Name: mpmat_index, dtype: int64

16832
<sample>
DetectSeq_ATP8-DddA11_REP-1      307
DetectSeq_ATP8-DddA6_REP-1       931
DetectSeq_ATP8-DddAwt_REP-1      145
DetectSeq_JAK2-DddA11_REP-1     1014
DetectSeq_JAK2-DddA11_REP-2      874
DetectSeq_SIRT6-DddA11_REP-1    3888
DetectSeq_SIRT6-DddA11_REP-2    9673
Name: mpmat_index, dtype: int64
0


In [15]:
print(df_sign_lenient.shape)  # (16832, 28)
df_sign_lenient.head(2)

(16832, 28)


,<sample>,chr_name,region_start,region_end,mpmat_index,region_site_num,region_block_site_num,region_mut_site_num,region_site_index,region_block_state,region_highest_site_index,region_highest_site_mut_num,region_highest_site_cover_num,region_highest_site_mut_ratio,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR
13,DetectSeq_ATP8-DddA11_REP-1,chr1,633244,633286,chr1_633244_633286,9,0,9,"chr1_633244_GA,chr1_633246_GA,chr1_633249_GA,c...",N-N-N-N-N-N-N-N-N,chr1_633286_GA,260,386,0.673575,22,942,0,168,0.055693,5.615117,0.0,1.001422,"0,1,2,3,4,5,6,7,8,9 19,3,0,0,0,0,0,0,0,0 564,1...",6.655668,6.949872,TestOK,2.429064e-61,1.530417e-57
629,DetectSeq_ATP8-DddA11_REP-1,chr1,18907111,18907133,chr1_18907111_18907133,9,0,9,"chr1_18907111_GA,chr1_18907119_GA,chr1_1890712...",N-N-N-N-N-N-N-N-N,chr1_18907131_GA,14,46,0.304348,28,63,0,35,0.070882,0.375533,0.0,0.208630,"0,1,2,3,4,5,6,7,8,9 28,0,0,0,0,0,0,0,0,0 16,12...",2.405442,4.686838,TestOK,3.002958e-11,2.855841e-08


# report putative off-target list (use df_sign_lenient), waiting for igv checking

In [16]:
# %%% use pois.index filter mpmat
mpmat_filename = "../mpmat/mpmat_all_C6M6R2_after_rmdup_rm_oldoverlap_rmblacklist_add_NatIndex_sort2_rmdup2.mpmat"

df_mpmat = pd.read_csv(mpmat_filename, header=None, index_col=None, sep='\t')
df_mpmat
df_mpmat.columns = ['chrom', 'start', 'end'] + df_mpmat.columns[3:].tolist()
df_mpmat

# add_mpmat_index
df_mpmat['mpmat_index'] = df_mpmat.apply(lambda x: '_'.join(x[['chrom', 'start', 'end']].astype(str)), axis=1)

# mpmat_select
os.makedirs('./final_list', exist_ok=True)

for sample, df_select in df_sign_lenient.groupby('<sample>'):
    df_mpmat_select = pd.merge(df_mpmat, df_select['mpmat_index'], how='right', on='mpmat_index').drop(columns='mpmat_index')
    df_mpmat_select.to_csv(f'../final_list/2023-04-20_{sample}_final_list.mpmat', header=False, index=False, sep='\t')

## generate igv_shot_script (Provide it to IGV for batch screenshots)

In [17]:
# %%% IGV snapshot check

df_sample = df_sign_lenient.loc[:, ['chr_name', 'region_start', 'region_end', 'mpmat_index', '<sample>']].copy()
df_sample

df_sample['mpmat_index'] = df_sample['<sample>'] + '_' + df_sample['mpmat_index']
df_sample

df_sample.groupby('<sample>').count()  # min 145  DetectSeq_ATP8-DddAwt_REP-1

# df_sample = df_sample.groupby('<sample>').sample(206)

df_igv = df_sample[['chr_name', 'region_start', 'region_end', 'mpmat_index']].copy()
print(df_igv)

#        chr_name  region_start  region_end                                        mpmat_index
# 13         chr1        633244      633286     DetectSeq_ATP8-DddA11_REP-1_chr1_633244_633286
# 629        chr1      18907111    18907133  DetectSeq_ATP8-DddA11_REP-1_chr1_18907111_1890...
# 751        chr1      22704608    22704644  DetectSeq_ATP8-DddA11_REP-1_chr1_22704608_2270...
# 931        chr1      27294796    27294894  DetectSeq_ATP8-DddA11_REP-1_chr1_27294796_2729...
# 1078       chr1      29332374    29332405  DetectSeq_ATP8-DddA11_REP-1_chr1_29332374_2933...
#         ...           ...         ...                                                ...
# 705609     chrX     154679199   154679238  DetectSeq_SIRT6-DddA11_REP-2_chrX_154679199_15...
# 705623     chrX     154981068   154981104  DetectSeq_SIRT6-DddA11_REP-2_chrX_154981068_15...
# 705624     chrX     155049205   155049215  DetectSeq_SIRT6-DddA11_REP-2_chrX_155049205_15...
# 705636     chrX     155216206   155216224  DetectSeq_SIRT6-DddA11_REP-2_chrX_155216206_15...
# 705643     chrX     155612557   155612564  DetectSeq_SIRT6-DddA11_REP-2_chrX_155612557_15...


# 填写相关信息
date = 20230421
# A local computer which installed an IGV app!!!!
output_path = f'/Volumes/zhaohn_HD/Bio/3.project/2022_DdCBE-3D-Genome_topic/2022-09-30_Detect-seq_batch-1_ATP8_JAK2_SIRT6/igv-snapshot_{date}'
fmt = "png"
height = 1500


# 格式化脚本
text = f"maxPanelHeight {height}\nsnapshotDirectory {output_path}/igv\n\n"

for index, row_info in df_igv.iterrows():
    chrom, start, stop, bed_name = row_info

    output_name = f'{bed_name}.snapshot.{fmt}'
    middle = int((start + stop) / 2)

    text += f"goto {chrom}:{middle - 80}-{middle + 80}\nsort position\nexpand\nviewaspairs\nsnapshot {output_name}\n\n"
print(text[:1000])

# for truly run
# os.makedirs(output_path, exist_ok=True)
# with open(f'{output_path}/{date}_off-targets_snapshot.igv_shot_script', 'wt') as f:
#     f.write(text)

       chr_name  region_start  region_end                                        mpmat_index
13         chr1        633244      633286     DetectSeq_ATP8-DddA11_REP-1_chr1_633244_633286
629        chr1      18907111    18907133  DetectSeq_ATP8-DddA11_REP-1_chr1_18907111_1890...
751        chr1      22704608    22704644  DetectSeq_ATP8-DddA11_REP-1_chr1_22704608_2270...
931        chr1      27294796    27294894  DetectSeq_ATP8-DddA11_REP-1_chr1_27294796_2729...
1078       chr1      29332374    29332405  DetectSeq_ATP8-DddA11_REP-1_chr1_29332374_2933...
...         ...           ...         ...                                                ...
705609     chrX     154679199   154679238  DetectSeq_SIRT6-DddA11_REP-2_chrX_154679199_15...
705623     chrX     154981068   154981104  DetectSeq_SIRT6-DddA11_REP-2_chrX_154981068_15...
705624     chrX     155049205   155049215  DetectSeq_SIRT6-DddA11_REP-2_chrX_155049205_15...
705636     chrX     155216206   155216224  DetectSeq_SIRT6-DddA11_REP-

## merge IGV results from Manual confirmation!
- positive: for truly off-target region
- negative: for FP off-target region (to be removed)

In [18]:
# %%% 整合IGV和人工check的结果
ls_p = glob('../igv-snapshot_20230421/igv/*/positive/*.png')
ls_n = glob('../igv-snapshot_20230421/igv/*/negative/*.png')

np = len(ls_p)
nn = len(ls_n)
total = np + nn

print(f'True Positive: {np} ({np/total* 100:.2f}%)')
print(f'False Positive: {nn} ({nn/total* 100:.2f}%)')
print(f'Total: {total}')

True Positive: 15397 (91.47%)
False Positive: 1435 (8.53%)
Total: 16832


In [19]:
ls_info = list(zip(ls_p, ['positive'] * len(ls_p))) + list(zip(ls_n, ['negative'] * len(ls_n)))
ls_info[:5]

[('../igv-snapshot_20230421/igv/DetectSeq_JAK2-DddA11_REP-1/positive/DetectSeq_JAK2-DddA11_REP-1_chr11_4637069_4637091.snapshot.png',
  'positive'),
 ('../igv-snapshot_20230421/igv/DetectSeq_JAK2-DddA11_REP-1/positive/DetectSeq_JAK2-DddA11_REP-1_chrX_21480066_21480087.snapshot.png',
  'positive'),
 ('../igv-snapshot_20230421/igv/DetectSeq_JAK2-DddA11_REP-1/positive/DetectSeq_JAK2-DddA11_REP-1_chr1_155086212_155086220.snapshot.png',
  'positive'),
 ('../igv-snapshot_20230421/igv/DetectSeq_JAK2-DddA11_REP-1/positive/DetectSeq_JAK2-DddA11_REP-1_chr10_123135275_123135285.snapshot.png',
  'positive'),
 ('../igv-snapshot_20230421/igv/DetectSeq_JAK2-DddA11_REP-1/positive/DetectSeq_JAK2-DddA11_REP-1_chr1_90057896_90057946.snapshot.png',
  'positive')]

In [20]:
df_igv_checked = pd.DataFrame(ls_info, columns=['file_name', 'signal'])
df_igv_checked

,file_name,signal
0,../igv-snapshot_20230421/igv/DetectSeq_JAK2-Dd...,positive
1,../igv-snapshot_20230421/igv/DetectSeq_JAK2-Dd...,positive
2,../igv-snapshot_20230421/igv/DetectSeq_JAK2-Dd...,positive
3,../igv-snapshot_20230421/igv/DetectSeq_JAK2-Dd...,positive
4,../igv-snapshot_20230421/igv/DetectSeq_JAK2-Dd...,positive
...,...,...
16827,../igv-snapshot_20230421/igv/DetectSeq_SIRT6-D...,negative
16828,../igv-snapshot_20230421/igv/DetectSeq_SIRT6-D...,negative
16829,../igv-snapshot_20230421/igv/DetectSeq_SIRT6-D...,negative
16830,../igv-snapshot_20230421/igv/DetectSeq_SIRT6-D...,negative


In [21]:
df_igv_checked['sample'] = df_igv_checked['file_name'].str.split('/').str[-1].str.split('_chr').str[0]
df_igv_checked['mpmat_index'] = df_igv_checked['file_name'].str.split('/').str[-1].str.split('_chr').str[1].str[:-13]
df_igv_checked['mpmat_index'] = 'chr' + df_igv_checked['mpmat_index']
df_igv_checked

,file_name,signal,sample,mpmat_index
0,../igv-snapshot_20230421/igv/DetectSeq_JAK2-Dd...,positive,DetectSeq_JAK2-DddA11_REP-1,chr11_4637069_4637091
1,../igv-snapshot_20230421/igv/DetectSeq_JAK2-Dd...,positive,DetectSeq_JAK2-DddA11_REP-1,chrX_21480066_21480087
2,../igv-snapshot_20230421/igv/DetectSeq_JAK2-Dd...,positive,DetectSeq_JAK2-DddA11_REP-1,chr1_155086212_155086220
3,../igv-snapshot_20230421/igv/DetectSeq_JAK2-Dd...,positive,DetectSeq_JAK2-DddA11_REP-1,chr10_123135275_123135285
4,../igv-snapshot_20230421/igv/DetectSeq_JAK2-Dd...,positive,DetectSeq_JAK2-DddA11_REP-1,chr1_90057896_90057946
...,...,...,...,...
16827,../igv-snapshot_20230421/igv/DetectSeq_SIRT6-D...,negative,DetectSeq_SIRT6-DddA11_REP-1,chr19_25179300_25179379
16828,../igv-snapshot_20230421/igv/DetectSeq_SIRT6-D...,negative,DetectSeq_SIRT6-DddA11_REP-1,chr17_6377964_6378003
16829,../igv-snapshot_20230421/igv/DetectSeq_SIRT6-D...,negative,DetectSeq_SIRT6-DddA11_REP-1,chr5_47889106_47889151
16830,../igv-snapshot_20230421/igv/DetectSeq_SIRT6-D...,negative,DetectSeq_SIRT6-DddA11_REP-1,chr1_124303789_124303855


# report final off-target list (final list)

In [22]:
# %%% use igv check results filter mpmat
mpmat_filename = "../mpmat/mpmat_all_C6M6R2_after_rmdup_rm_oldoverlap_rmblacklist_add_NatIndex_sort2_rmdup2.mpmat"

df_mpmat = pd.read_csv(mpmat_filename, header=None, index_col=None, sep='\t')
df_mpmat
df_mpmat.columns = ['chrom', 'start', 'end'] + df_mpmat.columns[3:].tolist()
df_mpmat

# add_mpmat_index
df_mpmat['mpmat_index'] = df_mpmat.apply(lambda x: '_'.join(x[['chrom', 'start', 'end']].astype(str)), axis=1)

# mpmat_select
os.makedirs('./final_list_after_igv_check', exist_ok=True)

for sample, df_select in df_igv_checked.query('signal=="positive"').groupby('sample'):
    df_mpmat_select = pd.merge(df_mpmat, df_select['mpmat_index'], how='right', on='mpmat_index').drop(columns='mpmat_index')
    df_mpmat_select.to_csv(f'../final_list_after_igv_check/2023-04-20_{sample}_final_list.mpmat', header=False, index=False, sep='\t')

# cd final_list_after_igv_check
# wc -l 2023-04-20*
  #   215 2023-04-20_DetectSeq_ATP8-DddA11_REP-1_final_list.mpmat
  #   772 2023-04-20_DetectSeq_ATP8-DddA6_REP-1_final_list.mpmat
  #    40 2023-04-20_DetectSeq_ATP8-DddAwt_REP-1_final_list.mpmat
  #   821 2023-04-20_DetectSeq_JAK2-DddA11_REP-1_final_list.mpmat
  #   757 2023-04-20_DetectSeq_JAK2-DddA11_REP-2_final_list.mpmat
  #  3399 2023-04-20_DetectSeq_SIRT6-DddA11_REP-1_final_list.mpmat
  #  9393 2023-04-20_DetectSeq_SIRT6-DddA11_REP-2_final_list.mpmat
  # 15397 total

## merge final list to one table and add poission test result

In [23]:
# %%% 已确认off-target list 基本统计
mpmats = sorted(glob('../final_list_after_igv_check/*DetectSeq*.mpmat'))
mpmats

tags = [i.split('DetectSeq_')[-1].split('_final_list')[0] for i in mpmats]
tags


print(mpmats)
print()
print(tags)
print()
mpmats = ','.join(mpmats)
tags = ','.join(tags)

print(mpmats)
print()
print(tags)
# !pip install bioat -U
# !bioat table merge -h
# !echo $mpmats
# !echo $tags
!bioat table merge $mpmats $tags ../final_list_after_igv_check/2023-04-24_merged_final_list.mpmat

['../final_list_after_igv_check/2023-04-20_DetectSeq_ATP8-DddA11_REP-1_final_list.mpmat', '../final_list_after_igv_check/2023-04-20_DetectSeq_ATP8-DddA6_REP-1_final_list.mpmat', '../final_list_after_igv_check/2023-04-20_DetectSeq_ATP8-DddAwt_REP-1_final_list.mpmat', '../final_list_after_igv_check/2023-04-20_DetectSeq_JAK2-DddA11_REP-1_final_list.mpmat', '../final_list_after_igv_check/2023-04-20_DetectSeq_JAK2-DddA11_REP-2_final_list.mpmat', '../final_list_after_igv_check/2023-04-20_DetectSeq_SIRT6-DddA11_REP-1_final_list.mpmat', '../final_list_after_igv_check/2023-04-20_DetectSeq_SIRT6-DddA11_REP-2_final_list.mpmat']

['ATP8-DddA11_REP-1', 'ATP8-DddA6_REP-1', 'ATP8-DddAwt_REP-1', 'JAK2-DddA11_REP-1', 'JAK2-DddA11_REP-2', 'SIRT6-DddA11_REP-1', 'SIRT6-DddA11_REP-2']

../final_list_after_igv_check/2023-04-20_DetectSeq_ATP8-DddA11_REP-1_final_list.mpmat,../final_list_after_igv_check/2023-04-20_DetectSeq_ATP8-DddA6_REP-1_final_list.mpmat,../final_list_after_igv_check/2023-04-20_DetectSeq_AT

In [24]:
# check merged table
df_merged_mpmat = pd.read_csv('../final_list_after_igv_check/2023-04-24_merged_final_list.mpmat',
                              header=None, sep='\t')
df_merged_mpmat.columns = [
    '<sample>',  # 0
    'chr_name',  # 1
    'region_start',  # 2
    'region_end',  # 3
    'region_site_num',  # 4
    'region_mut_site_num',  # 5
    'region_SNP_mut_num',  # 6
    'region_site_index',  # 7
    'mut_base_num',  # 8
    'cover_base_num',  # 9
    'mut_ratio',  # 10
    'SNP_ann',  # 11
    'tandem_info',  # 12
    'pass_info',  # 13
]
df_merged_mpmat

,<sample>,chr_name,region_start,region_end,region_site_num,region_mut_site_num,region_SNP_mut_num,region_site_index,mut_base_num,cover_base_num,mut_ratio,SNP_ann,tandem_info,pass_info
0,ATP8-DddA11_REP-1,chr11,28107123,28107196,15,14,0,"chr11_28107123_CT,chr11_28107128_CT,chr11_2810...","2,1,1,0,2,5,12,9,2,5,5,3,4,1,3","33,32,31,30,31,30,31,31,30,31,31,29,31,30,30","0.06061,0.03125,0.03226,0,0.06452,0.16667,0.38...","False,False,False,False,False,False,False,Fals...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0","Filter,Filter,Filter,Filter,Filter,Filter,Pass..."
1,ATP8-DddA11_REP-1,chr4,127523472,127523528,14,14,0,"chr4_127523472_CT,chr4_127523474_CT,chr4_12752...","6,4,3,10,6,3,7,7,3,5,4,8,3,1","27,26,25,26,26,26,28,29,30,29,30,29,30,28","0.22222,0.15385,0.12,0.38462,0.23077,0.11538,0...","False,False,False,False,False,False,False,Fals...","0,0,0,0,0,0,0,0,0,0,0,0,0,0","Pass,Filter,Filter,Pass,Pass,Filter,Pass,Pass,..."
2,ATP8-DddA11_REP-1,chr10,79996559,79996604,8,7,0,"chr10_79996559_CT,chr10_79996567_C.,chr10_7999...","1,0,6,9,5,3,3,3","18,21,21,21,20,20,19,20","0.05556,0,0.28571,0.42857,0.25,0.15,0.15789,0.15","False,False,False,False,False,False,False,False","0,0,0,0,0,0,0,0","Filter,Filter,Pass,Pass,Filter,Filter,Filter,F..."
3,ATP8-DddA11_REP-1,chr20,36808712,36808741,13,12,0,"chr20_36808712_CT,chr20_36808714_CT,chr20_3680...","6,7,4,2,3,2,4,4,2,1,2,0,2","22,19,20,22,21,22,21,23,25,25,24,23,24","0.27273,0.36842,0.2,0.09091,0.14286,0.09091,0....","False,False,False,False,False,False,False,Fals...","0,0,0,0,0,0,0,0,0,0,0,0,0","Pass,Pass,Filter,Filter,Filter,Filter,Filter,F..."
4,ATP8-DddA11_REP-1,chr9,115952242,115952303,12,12,0,"chr9_115952242_GA,chr9_115952244_GA,chr9_11595...","3,2,2,2,8,2,5,2,5,4,4,6","26,27,26,25,24,23,23,24,24,24,22,22","0.11538,0.07407,0.07692,0.08,0.33333,0.08696,0...","False,False,False,False,False,False,False,Fals...","0,0,0,0,0,0,0,0,0,0,0,0","Filter,Filter,Filter,Filter,Pass,Filter,Filter..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15392,SIRT6-DddA11_REP-2,chr1,148725246,148725266,7,7,0,"chr1_148725246_CT,chr1_148725253_CT,chr1_14872...","1,13,6,2,2,2,4","17,20,17,20,18,20,19","0.05882,0.65,0.35294,0.1,0.11111,0.1,0.21053","False,False,False,False,False,False,False","0,0,0,0,0,0,0","Filter,Pass,Pass,Filter,Filter,Filter,Filter"
15393,SIRT6-DddA11_REP-2,chr8,81158725,81158758,11,11,0,"chr8_81158725_GA,chr8_81158729_GA,chr8_8115873...","1,1,3,1,2,6,6,3,4,4,10","27,27,28,27,24,23,24,24,25,23,25","0.03704,0.03704,0.10714,0.03704,0.08333,0.2608...","False,False,False,False,False,False,False,Fals...","0,0,0,0,0,0,0,0,0,0,0","Filter,Filter,Filter,Filter,Filter,Pass,Pass,F..."
15394,SIRT6-DddA11_REP-2,chr8,100191816,100191830,7,7,0,"chr8_100191816_GA,chr8_100191817_GA,chr8_10019...","2,2,1,3,7,7,10","36,38,36,35,33,31,33","0.05556,0.05263,0.02778,0.08571,0.21212,0.2258...","False,False,False,False,False,False,False","0,0,0,0,0,0,0","Filter,Filter,Filter,Filter,Pass,Pass,Pass"
15395,SIRT6-DddA11_REP-2,chr17,32831978,32831991,7,7,0,"chr17_32831978_CT,chr17_32831979_CT,chr17_3283...","21,9,6,8,8,5,5","39,41,40,40,39,36,40","0.53846,0.21951,0.15,0.2,0.20513,0.13889,0.125","False,False,False,False,False,False,False","0,0,0,0,0,0,0","Pass,Pass,Pass,Pass,Pass,Filter,Filter"


In [25]:
df_merged_mpmat.to_csv('../final_list_after_igv_check/2023-04-24_merged_final_list_add_header.mpmat', index=False, sep='\t')

In [26]:
df_merged_mpmat

,<sample>,chr_name,region_start,region_end,region_site_num,region_mut_site_num,region_SNP_mut_num,region_site_index,mut_base_num,cover_base_num,mut_ratio,SNP_ann,tandem_info,pass_info
0,ATP8-DddA11_REP-1,chr11,28107123,28107196,15,14,0,"chr11_28107123_CT,chr11_28107128_CT,chr11_2810...","2,1,1,0,2,5,12,9,2,5,5,3,4,1,3","33,32,31,30,31,30,31,31,30,31,31,29,31,30,30","0.06061,0.03125,0.03226,0,0.06452,0.16667,0.38...","False,False,False,False,False,False,False,Fals...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0","Filter,Filter,Filter,Filter,Filter,Filter,Pass..."
1,ATP8-DddA11_REP-1,chr4,127523472,127523528,14,14,0,"chr4_127523472_CT,chr4_127523474_CT,chr4_12752...","6,4,3,10,6,3,7,7,3,5,4,8,3,1","27,26,25,26,26,26,28,29,30,29,30,29,30,28","0.22222,0.15385,0.12,0.38462,0.23077,0.11538,0...","False,False,False,False,False,False,False,Fals...","0,0,0,0,0,0,0,0,0,0,0,0,0,0","Pass,Filter,Filter,Pass,Pass,Filter,Pass,Pass,..."
2,ATP8-DddA11_REP-1,chr10,79996559,79996604,8,7,0,"chr10_79996559_CT,chr10_79996567_C.,chr10_7999...","1,0,6,9,5,3,3,3","18,21,21,21,20,20,19,20","0.05556,0,0.28571,0.42857,0.25,0.15,0.15789,0.15","False,False,False,False,False,False,False,False","0,0,0,0,0,0,0,0","Filter,Filter,Pass,Pass,Filter,Filter,Filter,F..."
3,ATP8-DddA11_REP-1,chr20,36808712,36808741,13,12,0,"chr20_36808712_CT,chr20_36808714_CT,chr20_3680...","6,7,4,2,3,2,4,4,2,1,2,0,2","22,19,20,22,21,22,21,23,25,25,24,23,24","0.27273,0.36842,0.2,0.09091,0.14286,0.09091,0....","False,False,False,False,False,False,False,Fals...","0,0,0,0,0,0,0,0,0,0,0,0,0","Pass,Pass,Filter,Filter,Filter,Filter,Filter,F..."
4,ATP8-DddA11_REP-1,chr9,115952242,115952303,12,12,0,"chr9_115952242_GA,chr9_115952244_GA,chr9_11595...","3,2,2,2,8,2,5,2,5,4,4,6","26,27,26,25,24,23,23,24,24,24,22,22","0.11538,0.07407,0.07692,0.08,0.33333,0.08696,0...","False,False,False,False,False,False,False,Fals...","0,0,0,0,0,0,0,0,0,0,0,0","Filter,Filter,Filter,Filter,Pass,Filter,Filter..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15392,SIRT6-DddA11_REP-2,chr1,148725246,148725266,7,7,0,"chr1_148725246_CT,chr1_148725253_CT,chr1_14872...","1,13,6,2,2,2,4","17,20,17,20,18,20,19","0.05882,0.65,0.35294,0.1,0.11111,0.1,0.21053","False,False,False,False,False,False,False","0,0,0,0,0,0,0","Filter,Pass,Pass,Filter,Filter,Filter,Filter"
15393,SIRT6-DddA11_REP-2,chr8,81158725,81158758,11,11,0,"chr8_81158725_GA,chr8_81158729_GA,chr8_8115873...","1,1,3,1,2,6,6,3,4,4,10","27,27,28,27,24,23,24,24,25,23,25","0.03704,0.03704,0.10714,0.03704,0.08333,0.2608...","False,False,False,False,False,False,False,Fals...","0,0,0,0,0,0,0,0,0,0,0","Filter,Filter,Filter,Filter,Filter,Pass,Pass,F..."
15394,SIRT6-DddA11_REP-2,chr8,100191816,100191830,7,7,0,"chr8_100191816_GA,chr8_100191817_GA,chr8_10019...","2,2,1,3,7,7,10","36,38,36,35,33,31,33","0.05556,0.05263,0.02778,0.08571,0.21212,0.2258...","False,False,False,False,False,False,False","0,0,0,0,0,0,0","Filter,Filter,Filter,Filter,Pass,Pass,Pass"
15395,SIRT6-DddA11_REP-2,chr17,32831978,32831991,7,7,0,"chr17_32831978_CT,chr17_32831979_CT,chr17_3283...","21,9,6,8,8,5,5","39,41,40,40,39,36,40","0.53846,0.21951,0.15,0.2,0.20513,0.13889,0.125","False,False,False,False,False,False,False","0,0,0,0,0,0,0","Pass,Pass,Pass,Pass,Pass,Filter,Filter"


In [27]:
# %%% get poisson result for final list
ls_pois_selects = []

for sample, df_select in df_igv_checked.query('signal=="positive"').groupby('sample'):
    df_sign_select = df_sign_lenient.query('`<sample>`==@sample')
    df_pois_select = pd.merge(df_sign_select, df_select['mpmat_index'], how='right', on='mpmat_index').drop(columns='mpmat_index')
    ls_pois_selects.append(df_pois_select)


df_pois_out = pd.concat(ls_pois_selects)
df_pois_out

df_pois_out.groupby('<sample>')['<sample>'].count()

df_pois_out.to_csv('../final_list_after_igv_check/2023-04-24_merged_final_list_add_header_poisson_result.csv', index=False)

In [28]:
# wc -l *
#      215 2023-04-20_DetectSeq_ATP8-DddA11_REP-1_final_list.mpmat
#      772 2023-04-20_DetectSeq_ATP8-DddA6_REP-1_final_list.mpmat
#       40 2023-04-20_DetectSeq_ATP8-DddAwt_REP-1_final_list.mpmat
#      821 2023-04-20_DetectSeq_JAK2-DddA11_REP-1_final_list.mpmat
#      757 2023-04-20_DetectSeq_JAK2-DddA11_REP-2_final_list.mpmat
#     3399 2023-04-20_DetectSeq_SIRT6-DddA11_REP-1_final_list.mpmat
#     9393 2023-04-20_DetectSeq_SIRT6-DddA11_REP-2_final_list.mpmat
#    15398 2023-04-24_merged_final_list_add_header.mpmat
#    15398 2023-04-24_merged_final_list_add_header_poisson_result.csv
#    15397 2023-04-24_merged_final_list.mpmat
#    61590 total